In [6]:
"""
Requires:
* pip install pandas
"""

import itertools
from collections import defaultdict
from dataclasses import dataclass
from typing import Optional

import pandas as pd

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

IS_MAIN = __name__ == '__main__'

In [7]:
STATUS_NAME = ['死', '活', '三尸']


@dataclass
class Status:
    round: Optional[int] = None
    status: Optional[tuple] = None  # 0死 1活且无三尸 2有三尸
    fires: Optional[tuple] = None

    def serialize(self):
        people = list(sorted(map(lambda x: "".join(map(str, x)), zip(self.status, self.fires))))
        return "".join(map(str, [self.round % 4] + people))

    @classmethod
    def deserialize(cls, data):
        round = int(data[0])
        status = tuple(int(data[i]) for i in [1, 3, 5])
        fires = tuple(int(data[i]) for i in [2, 4, 6])
        return cls(round, status, fires)


def stringify_step(step):
    return "".join(map(str, step))


def stringify_steps(steps):
    return "-".join(map(stringify_step, steps))


def to_next_state(round, status, fires, step, ability_index):
    """只有当到不了下一回合时，才返回None"""
    # 我方行动
    if any(step[i] == 1 and status[i] == 2 for i in range(3)):
        return None
    new_status = status[:]
    new_fires = fires[:]
    for i in range(3):
        if step[i] == 1:
            new_status[i] = 0
    # 如果不满足三尸条件，这就是最后一回合了
    end_cond2 = (round + sum(1 if s == 2 else 0 for s in new_status)) % 2 != 1
    if end_cond2:
        return None
    # 敌方行动
    # 张鲁技能
    if round == 0:
        if new_status[ability_index] == 0 or new_status[ability_index] != 2:
            return None
        new_fires[ability_index] = min(3, new_fires[ability_index] + 1)
    # 敌人技能
    for i in range(3):
        if new_status[i] == 0: continue
        if new_fires[i] == 3:
            # 如果这回合放技能
            if new_status[i] == 2:
                return None
            else:
                new_status[i] = 0
                new_fires[i] = 0
    # 进入下一回合，取消三尸
    for i in range(3):
        if new_status[i] == 2:
            new_status[i] = 1
    # 下一回合，回合开始时
    new_round = (round + 1) % 4
    # 加火
    for i in range(3):
        if new_status[i] != 0:
            new_fires[i] = min(3, new_fires[i] + 1)
    # 张鲁复活
    for i in range(3):
        if new_status[i] == 0:
            new_status[i] = 2
    # 如果张鲁这回合放技能，有人有三尸且2火，这局也完了
    # end_cond1 = new_round == 0 and any(new_status[i] == 2 and new_fires[i] == 2 for i in range(3))
    # if end_cond1:
    #     return None
    return Status(new_round, tuple(new_status), tuple(new_fires))


def add_edges(p):
    def get_step_state(step, status, fires):
        to_kill = []
        to_keep = []
        for i in range(3):
            st = f"{status[i]}{fires[i]}"
            if step[i] == 1:
                to_kill.append(st)
            else:
                to_keep.append(st)
        return "".join(map(str, sorted(to_kill))) + "-" + "".join(map(str, sorted(to_keep)))

    status, fires = list(p.status), list(p.fires)
    # 我方行动
    living_deads = [i for i in range(3) if status[i] == 2]
    visited = set()
    edges = defaultdict(lambda: defaultdict(float))
    for step in itertools.product([1, 0], repeat=3):
        st = get_step_state(step, status, fires)
        if st in visited:
            continue
        visited.add(st)
        to_check = [0] if p.round != 0 else living_deads
        for ability_index in to_check:
            next_p = to_next_state(p.round, status, fires, step, ability_index)
            if next_p is not None:
                next_pid = next_p.serialize()
                sid = stringify_step(step)
                prob = 1 / len(to_check)
                edges[sid][next_pid] += prob
    return edges


def build_graph(initial_point):
    to_add = {initial_point.serialize()}
    edges = {}
    while to_add:
        end_points = set()
        for node in to_add:
            if node not in edges:
                res = add_edges(Status.deserialize(node))
                edges[node] = res
                end_points |= {nxt for sid, ends in res.items() for nxt, _ in ends.items()}
        to_add = end_points - set(edges.keys())
    return edges


if IS_MAIN:
    start = Status(1, (1, 1, 1), (1, 1, 1))
    EDGES = build_graph(start)

In [8]:
def find_all_paths(g_edges):
    def dfs(start, steps, prob, path):
        indices = {f: i for i, f in enumerate(path)}
        sid = stringify_steps(steps)
        if start in indices:
            repeat_from = indices[start]
            before = '-'.join(path[:repeat_from])
            repeat_part = '-'.join(path[repeat_from:])
            return {sid: {f"{before}{'-' if repeat_from > 0 else ''}({repeat_part})": prob}}
        if len(path) >= 50:
            return {sid: {'-'.join(path): prob}}
        actions = g_edges[start]
        ans = defaultdict(lambda: defaultdict(float))
        for act, end_points in actions.items():
            for nxt, p in end_points.items():
                res = dfs(nxt, steps + [act], prob * p, path + [start])
                for s, paths in res.items():
                    for pa, pp in paths.items():
                        ans[s][pa] += pp
        return ans

    return dfs('1111111', [], 1, [])


if IS_MAIN:
    PATHS = find_all_paths(EDGES)

In [9]:
def evaluate_paths(paths):
    rows = []
    for steps, pas in paths.items():
        for pa, pr in pas.items():
            rows.append(dict(steps=steps, path=pa, prob=pr))
    df = pd.DataFrame(rows)
    df['path_length'] = df['path'].apply(lambda x: len(x))
    df['max_kill'] = df['steps'].apply(lambda x: max(map(lambda y: sum(map(int, y)), x.split('-'))))
    df['is_circular'] = df['path'].apply(lambda x: '(' in x)
    gp_cols = ['steps', 'max_kill']
    prob = df[gp_cols + ['prob']].groupby(gp_cols).sum()
    state_cnt = df[gp_cols + ['path']].groupby(gp_cols).count().rename(columns=dict(path='state_count'))
    sdf = prob.join(state_cnt).reset_index().sort_values(['state_count', 'prob'], ascending=False)
    display(sdf.head())
    allowed = set(sdf[sdf['prob'] == max(sdf['prob'])]['steps'])
    display(df[df['is_circular'] == False])
    # 最高赌赢概率的所有解法
    df = df[df['steps'].isin(allowed)].sort_values(by=['max_kill', 'path_length'])
    display(df.head())
    df = df.iloc[0]
    return df['steps'], df['path']


if IS_MAIN:
    steps, path = evaluate_paths(PATHS)

,steps,max_kill,prob,state_count
0,100-000-100-000-000-100-100-100-000-000-000-10...,1,0.666667,1
12,100-000-100-000-000-100-100-100-000-000-000-10...,2,0.666667,1
16,100-000-100-000-000-100-100-100-000-000-000-10...,2,0.666667,1
17,100-000-100-000-000-100-100-100-000-000-000-10...,3,0.666667,1
18,100-000-100-000-000-100-100-100-000-000-000-10...,2,0.666667,1


,steps,path,prob,path_length,max_kill,is_circular


,steps,path,prob,path_length,max_kill,is_circular
382,100-000-100-000-000-100-100-100-000-000-000-10...,1111111-2121221-3121313-0202022-1111213-212132...,0.666667,161,1,True
171,100-000-100-000-110-000-000-100-100-100-000-00...,1111111-2121221-3121313-0202022-1111213-220212...,0.666667,129,2,True
172,100-000-100-000-000-100-100-100-000-110-100-10...,1111111-2121221-3121313-0202022-1111213-(21213...,0.666667,137,2,True
127,100-000-100-000-110-000-000-100-100-100-000-11...,1111111-2121221-(3121313-0202022-1111213-22021...,0.666667,145,2,True
126,100-000-100-000-110-000-000-100-100-100-000-11...,1111111-2121221-3121313-0202022-1111213-220212...,0.666667,153,2,True


In [10]:
def viz_solution(steps, path):
    def parse_states(path):
        tokens = path.split('-')
        repeat_from = None
        states = []
        for i, token in enumerate(tokens):
            if token.startswith('('):
                repeat_from = i
                token = token[1:]
            elif token.endswith(')'):
                token = token[:-1]
            states.append(token)
        return repeat_from, states

    def stringify_person(s, f):
        return f"{STATUS_NAME[s]} (能量{f})"

    each_step = steps.split('-')
    repeat_from, states = parse_states(path)
    L = len(states) - repeat_from
    rows = []
    for i, (s, act) in enumerate(zip(states, each_step)):
        row = {'回合': i + 1}
        st = Status.deserialize(s)
        st2 = list(f"{stringify_person(s, f)}{' 【X】' if a == '1' else ''}" for s, f, a in zip(st.status, st.fires, act))
        for j, str_act in enumerate(st2):
            row[f"小怪{j + 1}"] = str_act
        rows.append(row)
    df = pd.DataFrame(rows)
    return repeat_from, df


if IS_MAIN:
    repeat_from, df = viz_solution(steps, path)
    df.to_csv('../../output_data/9_60.csv', index=False)
    print(f"Repeat from {repeat_from}")
    display(df)

Repeat from 16


,回合,小怪1,小怪2,小怪3
0,1,活 (能量1) 【X】,活 (能量1),活 (能量1)
1,2,活 (能量2),活 (能量2),三尸 (能量1)
2,3,活 (能量2) 【X】,活 (能量3),活 (能量3)
3,4,三尸 (能量0),三尸 (能量0),三尸 (能量2)
4,5,活 (能量1),活 (能量2),活 (能量3)
5,6,活 (能量2) 【X】,活 (能量3),三尸 (能量0)
6,7,活 (能量1) 【X】,三尸 (能量0),三尸 (能量2)
7,8,活 (能量1) 【X】,活 (能量3),三尸 (能量1)
8,9,活 (能量3),三尸 (能量0),三尸 (能量1)
9,10,活 (能量1),活 (能量2),三尸 (能量0)
